#주제
- **음식 분류기** (multi-class food classifier)
- **앱 아이디어:** 음식 사진 찍으면 뭐 먹는 중인지 알려주는 + 칼로리 추정
- **잠재 유저:** 다이어터
- **데이터셋:** Food-101 (tensorflow_datasets에도 있음)
- **추가 기능:** "이 음식 레시피 알려줘", "근처 식당 추천" 같은 부가 기능 붙이기 가능.

# 0. 필요한 패키지 import

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os

# 1. 데이터셋 로드

In [2]:
(train_ds, val_ds, test_ds), ds_info = tfds.load(
    'food101',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/incomplete.M6CC1Y_2.0.0/food101-train.tfrecord*...:   0%|         …

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/incomplete.M6CC1Y_2.0.0/food101-validation.tfrecord*...:   0%|    …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


# 데이터셋 준비

In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_batches = train_ds.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_batches = val_ds.map(preprocess).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_batches = test_ds.map(preprocess).batch(BATCH_SIZE).prefetch(AUTOTUNE)

# 모델 디자인

In [4]:
base_model = tf.keras.applications.EfficientNetV2B0(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze for now

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(101, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


#학습

In [ ]:
EPOCHS = 15
history = model.fit(
    train_batches,
    validation_data=val_batches,
    epochs=EPOCHS
)

Epoch 1/15
   8/1894 ━━━━━━━━━━━━━━━━━━━━ 58:43 2s/step - accuracy: 0.0000e+00 - loss: 5.8331

# 평가 및 시각화

In [ ]:
loss, acc = model.evaluate(test_batches)
print(f'\nTest Accuracy: {acc:.4f}')

# 시각화
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.title("Training Curve")
plt.show()

#모델 저장 및 활용

In [ ]:
model.save("food_classifier_model")

# 라벨 인덱스 → 음식 이름
label_names = ds_info.features['label'].names

def predict_image(img_path):
    img = tf.keras.utils.load_img(img_path, target_size=IMG_SIZE)
    img_array = tf.keras.utils.img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, 0)  # batch dimension

    preds = model.predict(img_array)
    top_class = np.argmax(preds)
    confidence = np.max(preds)

    print(f"예측 결과: {label_names[top_class]} ({confidence * 100:.2f}%)")
    return label_names[top_class], confidence

테스트 예측

In [ ]:
predict_image('your_food_image.jpg')